# Weather Database API Script

1. Create a new set of 2,000 random latitudes and longitudes
2. Get the nearest city using the citipy module
3. Perform an API call with OpenWeatherMap
4. Retrieve the following information from the API call:
    * Latitude and Longitude
    * Maximum temperature
    * Percent humidity
    * Percent cloudiness
    * Wind speed
    * Weather description
5. Add the data to a new DataRame column order
    * City, Country, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Current Description
6. Export the data to a CSV file (Weather_Database/WeatherPy_Database.csv)

In [14]:
# Import the dependencies that we need
import pandas as pd # for DataFrame creation and csv manipulation
import numpy as np # for random number packages
from citipy import citipy # for find the nearest city
import requests # for API handling
import time # for the delay timer
import sys

# API key for OpenWeather
sys.path.insert(0,'/Users/David/Git/World_Weather_Analysis') # Comment out this code if config is at same level
from config import weather_api_key

In [15]:
# 1. Create a new set of 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

# Unpack the lat_lngs into a list object
coordinates = list(lat_lngs)

In [16]:
# 2. Get the nearest city using the citipy module

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

736

In [17]:
# 3. Perform an API call with OpenWeatherMap

# Basic URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                        "Country": city_country,
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humidity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | chelno-vershiny
Processing Record 2 of Set 1 | husavik
Processing Record 3 of Set 1 | hualmay
Processing Record 4 of Set 1 | tasiilaq
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | sisimiut
Processing Record 7 of Set 1 | hobart
Processing Record 8 of Set 1 | katsuura
Processing Record 9 of Set 1 | goure
Processing Record 10 of Set 1 | upernavik
Processing Record 11 of Set 1 | butaritari
Processing Record 12 of Set 1 | lumby
Processing Record 13 of Set 1 | yumen
Processing Record 14 of Set 1 | manzil tamim
City not found. Skipping...
Processing Record 15 of Set 1 | lorengau
Processing Record 16 of Set 1 | kruisfontein
Processing Record 17 of Set 1 | bengkulu
Processing Record 18 of Set 1 | lompoc
Processing Record 19 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 20 of Set 1 | narsaq
Processing Record 21 of Set 1 | kapaa
Processing Record 22 of S

In [21]:
# 4. Add the data to a new DataRame column order
# City, Country, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed, Current Description
city_data_df = pd.DataFrame(city_data)
# Rename Description to "Current Description"
city_data_df.rename(columns={"Description":"Current Description"},inplace=True)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chelno-Vershiny,RU,54.4333,51.1000,24.91,91,100,11.12,light snow
1,Husavik,IS,66.0449,-17.3389,37.31,84,14,12.71,few clouds
2,Hualmay,PE,-11.0964,-77.6139,74.30,73,100,5.88,overcast clouds
3,Tasiilaq,GL,65.6145,-37.6368,28.56,96,100,13.80,snow
4,Punta Arenas,CL,-53.1500,-70.9167,50.11,66,75,33.38,broken clouds
5,Sisimiut,GL,66.9395,-53.6735,14.52,82,100,8.63,overcast clouds
6,Hobart,AU,-42.8794,147.3294,70.05,79,0,6.91,clear sky
7,Katsuura,JP,35.1333,140.3000,45.10,63,6,17.43,clear sky
8,Goure,NE,13.9835,10.2704,77.23,11,100,5.01,overcast clouds
9,Upernavik,GL,72.7868,-56.1549,-10.07,98,63,7.11,broken clouds
